<a href="https://colab.research.google.com/github/kdidi99/ml_heidelberg/blob/main/tree_methods_stubs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminaries

In [228]:
# import modules
import numpy as np
import random
from sklearn.datasets import load_digits
import matplotlib.pyplot as p

In [161]:
# base classes

class Node:
    pass

class Tree:
    def __init__(self):
        self.root = Node()
    
    def find_leaf(self, x):
        node = self.root
        while hasattr(node, "feature"):
            j = node.feature
            if x[j] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node

# Density Tree

In [162]:
class DensityTree(Tree):
    def __init__(self):
        super(DensityTree, self).__init__()
        
    def train(self, data, prior, n_min=20):
        '''
        data: the feature matrix for the digit under consideration
        prior: the prior probability of this digit
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        self.prior = prior
        N, D = data.shape
        D_try = int(np.sqrt(D)) # number of features to consider for each split decision

        # find and remember the tree's bounding box, 
        # i.e. the lower and upper limits of the training feature set
        m, M = np.min(data, axis=0), np.max(data, axis=0)
        self.box = m.copy(), M.copy()
        
        # identify invalid features and adjust the bounding box
        # (If m[j] == M[j] for some j, the bounding box has zero volume, 
        #  causing divide-by-zero errors later on. We must exclude these
        #  features from splitting and adjust the bounding box limits 
        #  such that invalid features have no effect on the volume.)
        valid_features   = np.where(m != M)[0]
        invalid_features = np.where(m == M)[0]
        M[invalid_features] = m[invalid_features] + 1

        # initialize the root node
        self.root.data = data
        self.root.box = m.copy(), M.copy()

        # build the tree
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min:
                # Call 'make_density_split_node()' with 'D_try' randomly selected 
                # indices from 'valid_features'. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                permutation = np.random.permutation(len(valid_features))[:D_try] #+
                feature_indices = valid_features[permutation] #+
                left, right = make_density_split_node(node, N, feature_indices) #+
                stack.append(left) #+
                stack.append(right) #+
            else:
                # Call 'make_density_leaf_node()' to turn 'node' into a leaf node.
                make_density_leaf_node(node, N) #+ ### ab "if"-Zeile fast gleicher code auch im Decision tree, wenn 
                                                    # Fehler auftritt muss wahrscheinlich beides korrigiert werden

    def predict(self, x):
        leaf = self.find_leaf(x)
        # return p(x | y) * p(y) if x is within the tree's bounding box 
        # and return 0 otherwise
        if (x >= leaf.box[0]).all() and (x <= leaf.box[1]).all(): #+
            return leaf.response*self.prior #+
        else: #+
            return 0 #+

In [163]:
def make_density_split_node(node, N, feature_indices):
    '''
    node: the node to be split
    N:    the total number of training instances for the current class
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape
    m, M = node.box

    # find best feature j (among 'feature_indices') and best threshold t for the split
    e_min = float("inf")
    j_min, t_min = None, None
    
    for j in feature_indices:
        # Hint: For each feature considered, first remove duplicate feature values using 
        # 'np.unique()'. Describe here why this is necessary:
        # This is necessary because the thresholds are placed in the middle of the gaps between
        # the values after sorting them. There is no gap between two equal values which is why
        # duplicates need to be removed. A threshold must not be equal to a feature, because 
        # it would not be possible to decide whether the feature goes to the left or the right node 
        data_unique = np.unique(node.data[:, j]) #+
        # Compute candidate thresholds
        data_unique = np.sort(data_unique) #+
        #tj = [(data_unique[i]+data_unique[i+1])/2 for i in range(len(data_unique)-1)] #+
        tj = (data_unique[:len(data_unique)-1] + np.roll(data_unique, -1)[:len(data_unique)-1]) / 2 #+
        
        # Illustration: for loop - hint: vectorized version is possible
        for t in tj:
            # compute the error for left child when splitting j at t 
            data_l = node.data[node.data[:,j] <= t] #+
            n_l = len(data_l) #+
            box_l = np.max(data_l, axis=0) - np.min(data_l, axis=0) #+
            box_l[box_l == 0] = 1 #+
            v_l = np.product(box_l) #+
            loo_err_l = (n_l / (N * v_l)) * ((n_l / N) - 2 * ((n_l - 1)/(N - 1))) #+

            # compute the error for right child when splitting j at t 
            data_r = node.data[node.data[:,j] > t] #+
            n_r = len(data_r) #+
            box_r = np.max(data_r, axis=0) - np.min(data_r, axis=0) #+
            box_r[box_r == 0] = 1 #+
            v_r = np.product(box_r) #+
            loo_err_r = (n_r / (N * v_r)) * ((n_r / N) - 2 * ((n_r - 1)/(N - 1))) #+

            # compute overall error (we want to minimize the error for both 
            # the left and the right child, so we minimize the sum)
            loo_error = loo_err_l + loo_err_r #+
            
            # choose the best threshold that
            if loo_error < e_min:
                e_min = loo_error #+
                j_min = j #+
                t_min = t #+

    # create children
    left = Node()
    right = Node()

    # initialize 'left' and 'right' with the data subsets and bounding boxes
    # according to the optimal split found above
    #left.data = [i for i in node.data if i[j_min]<t_min] #+ # store data in left node -- for subsequent splits
    left.data = node.data[node.data[:,j_min] < t_min] #+
    left.box = np.min(left.data, axis=0).copy(), np.max(left.data, axis=0).copy() #+ # store bounding box in left node
    #right.data = [i for i in node.data if i[j_min]>t_min] #+
    right.data = node.data[node.data[:,j_min] > t_min] #+
    right.box = np.min(right.data, axis=0).copy(), np.max(right.data, axis=0).copy() #+

    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left = left
    node.right = right
    node.feature = j_min #+
    node.threshold = t_min #+

    # return the children (to be placed on the stack)
    return left, right

In [164]:
def make_density_leaf_node(node, N):
    '''
    node: the node to become a leaf
    N:    the total number of training instances for the current class
    '''
    # compute and store leaf response
    n = node.data.shape[0] #+
    box = node.box[1]-node.box[0]
    box[box == 0] = 1
    v = np.product(box) #+
    node.response = n/(N*v)

# Decision Tree

In [217]:
class DecisionTree(Tree):
    def __init__(self):
        super(DecisionTree, self).__init__()
        
    def train(self, data, labels, n_min=20):
        '''
        data: the feature matrix for all digits
        labels: the corresponding ground-truth responses
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        N, D = data.shape
        D_try = int(np.sqrt(D)) # how many features to consider for each split decision

        # initialize the root node
        self.root.data = data
        self.root.labels = labels
        
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min and not node_is_pure(node):
                # Call 'make_decision_split_node()' with 'D_try' randomly selected 
                # feature indices. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                feature_indices = np.random.permutation(D)[:D_try] #+c
                left, right = make_decision_split_node(node, feature_indices) #+c+
                stack.append(left) #+c
                stack.append(right) #+c
            else:
                # Call 'make_decision_leaf_node()' to turn 'node' into a leaf node.
                make_decision_leaf_node(node) #+c+ ### ab "if"-Zeile praktisch kopiert vom Density tree, wenn
                                                    # Fehler auftritt muss wahrscheinlich beides korrigiert werden
                
    def predict(self, x):
        leaf = self.find_leaf(x)
        # compute p(y | x)
        index = np.argmax(leaf.response, axis=0) #+
        y = leaf.response[index[0],1].astype(int)
        return y #+

In [181]:
def make_decision_split_node(node, feature_indices):
    '''
    node: the node to be split
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape

    # find best feature j (among 'feature_indices') and best threshold t for the split
    gini_min = float("inf") #+c
    j_min, t_min = None, None #+c

    for j in feature_indices: #+c
        data_unique = np.unique(node.data[:, j]) #+c
        # Compute candidate thresholds
        data_unique = np.sort(data_unique) #+c
        tj = (data_unique[:len(data_unique)-1] + np.roll(data_unique, -1)[:len(data_unique)-1]) / 2 #+c

        for t in tj: #+c
            # compute the gini impurity for left child when splitting j at t 
            data_l = node.data[node.data[:,j] <= t] #+c
            labels_l = node.labels[node.data[:,j] <= t] #+
            ulabels_l = np.unique(labels_l) #+
            n_l = len(data_l) #+c
            n_lk = np.zeros(len(ulabels_l)) #+

            for k in range(len(ulabels_l)): #+
                n_lk[k] = np.sum(labels_l == ulabels_l[k]) #+
            
            gini_l = n_l * (1 - np.sum(np.square(n_lk) / (n_l ** 2))) #+

            # compute the gini impurity for right child when splitting j at t 
            data_r = node.data[node.data[:,j] > t] #+c
            labels_r = node.labels[node.data[:,j] > t] #+
            ulabels_r = np.unique(labels_r) #+
            n_r = len(data_r) #+c
            n_rk = np.zeros(len(ulabels_r)) #+

            for k in range(len(ulabels_r)): #+
                n_rk[k] = np.sum(labels_r == ulabels_r[k]) #+
            
            gini_r = n_r * (1 - np.sum(np.square(n_rk) / (n_r ** 2))) #+
            
            # compute overall gini impurity (we want to minimize it for both 
            # the left and the right child, so we minimize the sum)
            gini = gini_l + gini_r #+

            if gini < gini_min: #+c
                gini_min = gini #+c
                j_min = j #+c
                t_min = t #+c

    # create children
    left = Node()
    right = Node()
    
    # initialize 'left' and 'right' with the data subsets and labels
    # according to the optimal split found above
    left.data = node.data[node.data[:,j_min] < t_min] #+c
    left.labels = node.labels[node.data[:,j_min] < t_min] #+
    #left.labels = [node.labels[i] for i in range(len(n)) if node.data[i][j_min]<t_min] #+ # corresponding labels
    right.data = node.data[node.data[:,j_min] > t_min] #+c
    right.labels = node.labels[node.data[:,j_min] > t_min] #+
    #right.labels = [node.labels[i] for i in range(len(n)) if node.data[i][j_min]>t_min] #+

    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left = left
    node.right = right
    node.feature = j_min #+c
    node.threshold = t_min #+c    

    # return the children (to be placed on the stack)
    return left, right    

In [180]:
def make_decision_leaf_node(node):
    '''
    node: the node to become a leaf
    '''
    # compute and store leaf response
    node.N = len(node.labels) #+

    unique = np.unique(node.labels)
    node.response = np.empty((len(unique),2)) #+ liste mit elementen [p(label), label]

    for i in range(len(unique)): #+
        node.response[i, 0] = np.sum(node.labels == unique[i])/node.N #+
        node.response[i, 1] = unique[i] #+
    
    #node.response = np.array([np.array([np.sum(node.labels == i)/node.N, i]) for i in np.unique(node.labels)])

In [183]:
def node_is_pure(node):
    '''
    check if 'node' ontains only instances of the same digit
    '''
    return len(np.unique(node.labels)) == 1 #+

# Evaluation of Density and Decision Tree

In [184]:
# read and prepare the digits data
digits = load_digits() 
data = digits["data"]
images = digits["images"]
target = digits["target"]
target_names = digits["target_names"]

X_all = data
y_all = target

y_unique = np.unique(target)
X_group = np.zeros(len(y_unique), dtype=object)
y_group = np.zeros(len(y_unique), dtype=object)

for i in range(len(y_unique)):
    X_group[i] = data[target == y_unique[i]]
    y_group[i] = target[target == y_unique[i]]


In [223]:
# train trees, plot training error confusion matrices, and comment on your results

# train density trees
den_trees = np.empty(len(y_unique), dtype=object)

for i in range(len(y_unique)):
    den_trees[i] = DensityTree()
    den_trees[i].train(X_group[i], 1/10, 20)

# train decision tree
dec_tree = DecisionTree()
dec_tree.train(X_all, y_all, 20)

In [233]:
# test both trees

def den_trees_predict(x):
    predictions = np.empty(len(y_unique))
    for i in range(len(y_unique)):
        predictions[i] = den_trees[i].predict(x)
    return y_unique[np.argmax(predictions)]

print(f"True: {y_all[13]}, Predict: {den_trees_predict(X_all[13])}")
print(f"True: {y_all[13]}, Predict: {dec_tree.predict(X_all[13])}")

den_tree_result = np.empty(len(y_all))
dec_tree_result = np.empty(len(y_all))

for i in range(len(X_all)):
    den_tree_result[i] = den_trees_predict(X_all[i])  
    dec_tree_result[i] = dec_tree.predict(X_all[i])   


True: 3, Predict: 3
True: 3, Predict: 3


In [237]:
# useful function to compute and print out the confusion matrix
def confusion_matrix(predicted, true, print_result = True):
    # all possible classes in training and testing data
    classes = np.unique(np.concatenate([true, predicted]))

    # for each matrix entry in row c_t, column c_p, count
    # for how many examples is the true label c_t and the predicted label c_p.
    matrix = [[np.sum((true==c_t) * (predicted==c_p)) / np.sum(true==c_t) for c_t in classes] for c_p in classes]

    if print_result:
        print('Class | ' +' |'.join(['%5d'%(c) for c in classes]))
        for c, row in zip(classes, matrix):
            print('------+-' + '+'.join(['------' for c in classes]))
            print('%5d | '%(c) + '|'.join(['%5.1f%%'%(100.*r) for r in row]))
        print()

    return matrix

print("Training error confusion matrix for Density Tree:\n")
mat = confusion_matrix(den_tree_result, y_all)
print("\nTraining error confusion matrix for Decision Tree:\n")
mat = confusion_matrix(dec_tree_result, y_all)

Training error confusion matrix for Density Tree:

Class |     0 |    1 |    2 |    3 |    4 |    5 |    6 |    7 |    8 |    9
------+-------+------+------+------+------+------+------+------+------+------
    0 | 100.0%|  0.0%|  0.0%|  0.0%|  0.0%|  0.0%|  0.0%|  0.0%|  0.0%|  0.0%
------+-------+------+------+------+------+------+------+------+------+------
    1 |   0.0%|100.0%|  0.0%|  0.0%|  0.0%|  0.0%|  0.0%|  0.0%|  0.0%|  0.0%
------+-------+------+------+------+------+------+------+------+------+------
    2 |   0.0%|  0.0%|100.0%|  0.0%|  0.0%|  0.0%|  0.0%|  0.0%|  0.0%|  0.0%
------+-------+------+------+------+------+------+------+------+------+------
    3 |   0.0%|  0.0%|  0.0%|100.0%|  0.0%|  0.0%|  0.0%|  0.0%|  0.0%|  0.0%
------+-------+------+------+------+------+------+------+------+------+------
    4 |   0.0%|  0.0%|  0.0%|  0.0%|100.0%|  0.0%|  0.0%|  0.0%|  0.0%|  0.0%
------+-------+------+------+------+------+------+------+------+------+------
    5 |   0.0%

# Density and Decision Forest

In [ ]:
class DensityForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DensityTree() for i in range(n_trees)]
    
    def train(self, data, prior, n_min=20):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            ... # your code here

    def predict(self, x):
        # compute the ensemble prediction
        return ... # your code here

In [ ]:
class DecisionForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DecisionTree() for i in range(n_trees)]
    
    def train(self, data, labels, n_min=0):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            ... # your code here

    def predict(self, x):
        # compute the ensemble prediction
        return ... # your code here

# Evaluation of Density and Decision Forest

In [ ]:
# train forests (with 20 trees per forest), plot training error confusion matrices, and comment on your results
... # your code here